In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score
import numpy as np # linear algebra
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, RFECV, SelectFromModel
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train=pd.read_csv("train.csv")
train_data=train.copy()
train_data
X=train_data
y=train_data.pop("SalePrice")
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=123)
x_num_columns=X.select_dtypes(include="number").copy().columns
x_cat_columns=X.select_dtypes(exclude="number").copy().columns
num_pipeline=make_pipeline(
    SimpleImputer(strategy="mean"), 
    StandardScaler()
)

cat_pipeline=make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"), 
    OneHotEncoder(handle_unknown="ignore")

)
preprocessor=ColumnTransformer(
    transformers=[
        ("num", num_pipeline, x_num_columns), 
        ("cat", cat_pipeline, x_cat_columns)
])

feature_selection=SelectKBest(score_func=f_regression, k=10)

model=RandomForestRegressor()

full_pipeline=make_pipeline(preprocessor, feature_selection, model)
full_pipeline.fit(X_train, y_train)
rmse_score = mean_squared_log_error(y_train, full_pipeline.predict(X_train))
rmse_score

r2_score(y_train, full_pipeline.predict(X_train))

submission=pd.read_csv("test.csv")
my_first_submission=(
    submission
        .assign(SalePrice=full_pipeline.predict(submission))
        .filter(["Id", "SalePrice"])
        .to_csv("my_first_submission.csv", index=False)
)